# Setup

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

print(tf.__version__)
print(np.__version__)

1.4.0
1.16.2


# Tensor Values
The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's rank is its number of dimensions, while its shape is a tuple of integers specifying the array's length along each dimension. Here are some examples of tensor values:
```python
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```
# TensorFlow Core Walkthrough
You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph (a `tf.Graph`).
2. Running the computational graph (using a `tf.Session`).

## Graph
A computational graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

- tf.Operation (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
- tf.Tensor: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.
Important: tf.Tensors do not have values, they are just handles to elements in the computation graph.

Let's build a simple computational graph. The most basic operation is a constant. The Python function that builds the operation takes a tensor value as input. The resulting operation takes no inputs. When run, it outputs the value that was passed to the constructor. We can create two floating point constants a and b as follows:

In [6]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)  # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [7]:
# writer = tf.summary.FileWriter('.')
# writer.add_graph(tf.get_default_graph())
# writer.flush()

## TensorBoard
TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.
First you save the computation graph to a TensorBoard summary file as follows:

This will produce an event file in the current directory with a name in the following format:
```
events.out.tfevents.{timestamp}.{hostname}
```
Now, in a new terminal, launch TensorBoard with the following shell command:

In [8]:
# ! tensorboard --logdir .

Then open TensorBoard's graphs page in your browser, and you should see a graph similar to the following:
![](https://camo.githubusercontent.com/56df99992f0a59e47d3c4e2ccee5f56c8a030654df820bcfb1144ac6e088c53d/68747470733a2f2f7777772e74656e736f72666c6f772e6f72672f696d616765732f67657474696e675f737461727465645f6164642e706e67)

# Session
To evaluate tensors, instantiate a `tf.Session` object, informally known as a session. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a `tf.Graph` is like a `.py` file, a `tf.Session` is like the `python` executable.
The following code creates a `tf.Session` object and then invokes its `run` method to evaluate the `total` tensor we created above:

In [9]:
sess = tf.Session()
print(sess.run(total))

7.0


When you request the output of a node with `Session.run` TensorFlow backtracks through the graph and runs all the nodes that provide input to the requested output node. So this prints the expected value of 7.0.

You can pass multiple tensors to `tf.Session.run`. The `run` method transparently handles any combination of tuples or dictionaries, as in the following example:

In [10]:
print(sess.run({'ab': (a, b), 'total': total}))

{'ab': (3.0, 4.0), 'total': 7.0}


During a call to `tf.Session.run` any `tf.Tensor` only has a single value. For example, the following code calls `tf.random_uniform` to produce a `tf.Tensor` that generates a random 3-element vector (with values in [0,1)):

In [11]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.7256386  0.21094108 0.51548314]
[0.8013824 0.3760885 0.8900175]
(array([1.4325136, 1.0244038, 1.8172659], dtype=float32), array([2.4325137, 2.0244038, 2.817266 ], dtype=float32))


Some TensorFlow functions return tf.Operations instead of tf.Tensors. The result of calling run on an Operation is None. You run an operation to cause a side-effect, not to retrieve a value. Examples of this include the initialization, and training ops demonstrated later.

## Feeding
As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later, like a function argument.

In [12]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

The preceding three lines are a bit like a function in which we define two input parameters (`x` and `y`) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument of the `tf.Session.run` method to feed concrete values to the placeholders:

In [13]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


Also note that the `feed_dict` argument can be used to overwrite any tensor in the graph. The only difference between placeholders and other `tf.Tensors `is that placeholders throw an error if no value is fed to them.

## Datasets
Placeholders work for simple experiments, but `tf.data` are the preferred method of streaming data into a model.

To get a runnable `tf.Tensor` from a Dataset you must first convert it to a `tf.data.Iterator`, and then call the Iterator's `tf.data.Iterator.get_next` method.

The simplest way to create an Iterator is with the `tf.data.Dataset.make_one_shot_iterator` method. For example, in the following code the `next_item` tensor will return a row from the `my_data` array on each `run` call:

In [14]:
my_data = [
    [0, 1, ],
    [2, 3, ],
    [4, 5, ],
    [6, 7, ],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

Reaching the end of the data stream causes Dataset to throw an `tf.errors.OutOfRangeError`. For example, the following code reads the `next_item` until there is no more data to read:

In [15]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


If the `Dataset` depends on stateful operations you may need to initialize the iterator before using it, as shown below:

In [16]:
r = tf.random_normal([10, 3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break


[ 0.06986391  1.4610301  -1.0036495 ]
[-0.7883714   0.1810219  -0.08442058]
[ 0.5085128  -2.2415664   0.42563212]
[ 0.0265535  1.4616607 -0.7144696]
[-1.566601    0.09507536 -0.19432232]
[0.05365401 0.6103887  0.7274601 ]
[ 0.19288394  0.82535994 -0.17944159]
[ 0.82877403 -1.1562536  -0.32464135]
[-0.1984801   0.01974896  0.7166367 ]
[1.3157581  1.6694859  0.14281015]


## Layers
A trainable model must modify the values in the graph to get new outputs with the same input. `tf.layers` are the preferred way to add trainable parameters to a graph.
Layers package together both the variables and the operations that act on them. For example a densely-connected layer performs a weighted sum across all inputs for each output and applies an optional activation function. The connection weights and biases are managed by the layer object.

### Creating Layers
The following code creates a `tf.layers.Dense` layer that takes a batch of input vectors, and produces a single output value for each. To apply a layer to an input, call the layer as if it were a function. For example:

In [17]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

The layer inspects its input to determine sizes for its internal variables. <mark>So here we must set the shape of the `x` placeholder so that the layer can build a weight matrix of the correct size.</mark>
Now that we have defined the calculation of the output, `y`, there is one more detail we need to take care of before we run the calculation.

### Initializing Layers
The layer contains variables that must be **initialized** before they can be used. While it is possible to initialize variables individually, you can easily initialize all the variables in a TensorFlow graph as follows:

In [18]:
init = tf.global_variables_initializer()
sess.run(init)

Important: Calling `tf.global_variables_initializer` only creates and returns a handle to a TensorFlow operation. That op will initialize all the global variables when we run it with `tf.Session.run`.
Also note that this `global_variables_initializer` only initializes variables that existed in the graph when the initializer was created. <mark>So the initializer should be one of the last things added during graph construction.</mark>

### Executing Layers
Now that the layer is initialized, we can evaluate the `linear_model`'s output tensor as we would any other tensor. For example, the following code:

In [19]:
print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-1.2446142]
 [-1.1509535]]


### Layer Function shortcuts
For each layer class (like `tf.layers.Dense`) TensorFlow also supplies a shortcut function (like `tf.layers.dense`). The only difference is that the <mark>shortcut function versions create and run the layer in a single call.</mark> For example, the following code is equivalent to the earlier version:

In [20]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))
sess.close()

[[1.1759804 ]
 [0.60365844]]


While convenient, this approach allows no access to the `tf.layers.Layer `object. This makes introspection and debugging more difficult, and layer reuse impossible.

## Feature columns
The easiest way to experiment with feature columns is using the `tf.feature_column.input_layer` function. This function only accepts dense columns as inputs, so to view the result of a categorical column you must wrap it in an `tf.feature_column.indicator_column`. For example:

In [21]:
features = {
    'sales': [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Running the `inputs` tensor will parse the `features` into a batch of vectors.

Feature columns can have internal state, like layers, so they often need to be initialized. Categorical columns use `tf.contrib.lookup` internally and these require a separate initialization op, `tf.tables_initializer`.

In [22]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

Once the internal state has been initialized you can run `inputs` like any other `tf.Tensor`:

In [23]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


This shows how the feature columns have packed the input vectors, with the one-hot "department" as the first two indices and "sales" as the third.

[For more information](https://zhuanlan.zhihu.com/p/73701872)

## Training
Now that you're familiar with the basics of core TensorFlow, let's train a small regression model manually.

### Define the data
First let's define some inputs, `x`, and the expected output for each input, `y_true`:

In [24]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### Define the model
Next, build a simple linear model, with 1 output:

In [25]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

You can evaluate the predictions as follows:

In [26]:
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[0.28113234]
 [0.5622647 ]
 [0.843397  ]
 [1.1245294 ]]


### Loss
To optimize a model, you first need to define the loss. We'll use the mean square error, a standard loss for regression problems.

While you could do this manually with lower level math operations, the t`f.losses` module provides a set of common loss functions. You can use it to calculate the mean square error as follows:

In [27]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

6.904089


### Training
TensorFlow provides **optimizers** implementing standard optimization algorithms. These are implemented as sub-classes of tf.train.Optimizer. They incrementally change each variable in order to minimize the loss. The simplest optimization algorithm is **gradient descent**, implemented by `tf.train.GradientDescentOptimizer`. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. For example:

In [28]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

This code builds all the graph components necessary for the optimization, and returns a training operation. When run, the training op will update variables in the graph. You might run it as follows:

In [29]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

6.904089
4.8736143
3.4642143
2.4857686
1.8063554
1.3344371
1.0064987
0.77846754
0.6197626
0.5091648
0.43195027
0.37790242
0.33993238
0.31312105
0.29405546
0.2803671
0.2704127
0.263052
0.25749356
0.25318846
0.2497557
0.24693093
0.24453063
0.24242753
0.2405333
0.23878649
0.23714468
0.23557821
0.23406658
0.23259556
0.23115528
0.22973879
0.22834131
0.2269595
0.22559103
0.22423431
0.22288811
0.22155163
0.22022435
0.21890584
0.2175957
0.21629386
0.21500003
0.21371421
0.21243611
0.21116582
0.20990315
0.20864801
0.20740053
0.20616043
0.20492783
0.20370255
0.20248464
0.201274
0.20007065
0.1988744
0.19768533
0.19650342
0.19532853
0.1941607
0.19299987
0.19184595
0.19069892
0.18955877
0.18842542
0.18729883
0.18617904
0.1850659
0.18395942
0.18285957
0.18176627
0.18067954
0.17959926
0.17852548
0.17745808
0.17639711
0.17534244
0.17429411
0.17325205
0.17221615
0.17118654
0.17016304
0.16914564
0.16813436
0.16712913
0.16612986
0.16513664
0.1641493
0.16316786
0.16219231
0.16122259
0.16025868
0.15930054
0

Since `train` is an op, not a tensor, it doesn't return a value when run. To see the progression of the loss during training, we run the loss tensor at the same time, producing output like the following:

### Complete program

In [31]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

# Define the model
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

with tf.Session() as sess:
    # init variables
    init = tf.global_variables_initializer()
    sess.run(init)
    # loss function
    loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(loss)
    for i in range(100):
        _, loss_value = sess.run((train, loss))
        print(loss_value)



4.789718
3.35233
2.3547835
1.6624365
1.1818612
0.84823054
0.61656314
0.4556464
0.34382337
0.2660664
0.2119481
0.17423318
0.14790118
0.12946846
0.11651789
0.1073722
0.10086766
0.09619664
0.092798844
0.09028545
0.08838663
0.086915195
0.08574123
0.08477457
0.08395268
0.08323214
0.082582824
0.08198383
0.081420645
0.08088317
0.08036441
0.07985952
0.079365134
0.078878865
0.07839914
0.07792473
0.07745491
0.07698909
0.07652691
0.076068036
0.07561234
0.07515961
0.07470983
0.074262835
0.07381864
0.07337712
0.07293832
0.07250215
0.072068624
0.07163769
0.071209356
0.0707836
0.07036037
0.06993972
0.06952152
0.06910587
0.0686927
0.06828198
0.06787375
0.06746793
0.06706455
0.06666357
0.06626502
0.065868825
0.065475
0.06508354
0.06469442
0.06430761
0.06392312
0.06354095
0.063161045
0.06278343
0.062408026
0.062034916
0.061664008
0.061295357
0.06092885
0.060564578
0.060202476
0.05984255
0.05948474
0.05912909
0.058775567
0.058424167
0.058074858
0.05772764
0.057382505
0.057039395
0.056698382
0.056359403
0